### AI HUB에서 한국어-중국어 번역 말뭉치(사회과학, 기술과학) 다운로드
+ 데이터 구축분야
    + 사회과학 : 금융/증시, 사회/노동/복지, 교육, 문화재/향토/K-FOOD, 조례, 정치/행정, K-POP(한류)/대중문화_공연_콘텐츠
    + 기술과학 : 의료/보건, 특허/기술, 자동차/교통/소재, IT/컴퓨터/모바일
+ 한글 원문 어절 수 : 평균 15어절
+ 수량 : 병렬 말뭉치 130만개씩 총 260만개
+ 분야별 세부 구축 수량(단위:만)
    + 사회과학 : 금융/증시(20만), 사회/노동/복지(20만), 교육(10만), 문화재/향토/K-FOOD(15만), 조례(20만), 정치/행정(25만), K-POP(한류)/대중문화_공연_콘텐츠(20만)
    + 기술과학 : 의료/보건(25만), 특허/기술(15만), 자동차/교통/소재(30만), IT/컴퓨터/모바일(60만)
    
<img src='https://aihub.or.kr/sites/default/files/styles/max_2600x2600/public/2021-05/028.%E1%84%92%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AE%E1%86%A8%E1%84%8B%E1%85%A5-%E1%84%8C%E1%85%AE%E1%86%BC%E1%84%80%E1%85%AE%E1%86%A8%E1%84%8B%E1%85%A5%20%E1%84%87%E1%85%A5%E1%86%AB%E1%84%8B%E1%85%A7%E1%86%A8%20%E1%84%86%E1%85%A1%E1%86%AF%E1%84%86%E1%85%AE%E1%86%BC%E1%84%8E%E1%85%B5%28%E1%84%89%E1%85%A1%E1%84%92%E1%85%AC%E1%84%80%E1%85%AA%E1%84%92%E1%85%A1%E1%86%A8%29_%E1%84%83%E1%85%A2%E1%84%91%E1%85%AD%E1%84%83%E1%85%A9%E1%84%86%E1%85%A7%E1%86%AB.png?itok=AQ_3oXxE' width='700' height='200'>

=> 9가지 정보 중 한국어, 중국어 정보만 추출

In [1]:
#!pip install jieba   

In [2]:
import os
import glob
import pandas as pd
import re
import jieba        #중국어 문장을 단어 단위로 분할 출력

In [3]:
path = os.getcwd()
train_files = glob.glob(os.path.join(path+'/Training', '*.csv'))
valid_files = glob.glob(os.path.join(path+'/Validation', '*.csv'))

x_train, y_train, x_valid, y_valid = [], [], [], []

for t in train_files:
    train_df = pd.read_csv(t)
    x_train.extend(train_df['중국어'].values)
    y_train.extend(train_df['한국어'].values)
    
for v in valid_files:
    valid_df = pd.read_csv(v)
    x_valid.extend(valid_df['중국어'].values)
    y_valid.extend(valid_df['한국어'].values)
    
x_train_s = pd.Series(x_train)
y_train_s = pd.Series(y_train)
train_df = pd.concat([x_train_s, y_train_s], axis=1)
train_df.columns = ['중국어', '한국어']

x_valid_s = pd.Series(x_valid)
y_valid_s = pd.Series(y_valid)
valid_df = pd.concat([x_valid_s, y_valid_s], axis=1)
valid_df.columns = ['중국어', '한국어']

print('번역 데이터 : 학습용 {}개, 검증용 {}개'.format(len(train_df), len(valid_df)))
train_df.sample(5)

C:\Users\yeonok\AppData\Local\Temp\ipykernel_14920\1557320390.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(t)


번역 데이터 : 학습용 2080000개, 검증용 260000개


,중국어,한국어
1759251,"随着在""曹国局面""上与共同民主党的支持率差距被缩小,黄代表逐渐陷入困境。",'조국 국면'에서 바짝 좁혀졌던 더불어민주당과의 지지율 격차까지 제자리로 퇴보하면서...
971633,"在东南亚市场处于重要地位的泰国,在Apple App Store的销售排名也上升到了第3位,...",동남아 시장에서 중요한 위치에 있는 태국에서도 애플 앱스토어 매출 순위 3위에 오르...
1712079,在利用最小信号开销来增加带宽效率的以转播局(RS: relay station)为基础的蜂窝...,최소의 신호 오버헤드로 대역폭 효율을 증가시키는 중계국(RS: relay stati...
428609,"金社长表示：""对于已经离开的会长,人们对他的功与过给予了很多不同的评价。我斗个胆说,如今像会...","김 사장은 ""떠나신 회장님 공과에 대해 여러 다른 평가가 있다""면서 ""감히 말씀 드..."
1580115,"警方的立场是,7名死者中有3名男性和1名女性的身份已经确认,但其余3人的身份要等到DNA检查...",경찰은 사망자 7명 중 남자 3명과 여자 1명의 신원은 확인했으나 나머지 3명에 대...


In [4]:
#한국어 전처리
"""
(전)           명절 가사 노동이 여성에게만 쏠리는 현상은 '성차별'이라는 지적이 있다.
(후)   <start> 명절 가사 노동이 여성에게만 쏠리는 현상은 ' 성차별 ' 이라는 지적이 있다 <end>
"""
def preprocess_kr(w):
    #?'!¿-·" 앞뒤로 공백 삽입
    w = re.sub(r"([?'!¿\-·\"])", r" \1 ", w)

    # (a-z, A-Z, .?!,)을 제외한 모든 것을 공백으로 대체
    w = re.sub(r'[ |ㄱ-ㅎ|ㅏ-ㅣ]+', r" ", w)

    #1000단위로 나누기 위한 , 는 그대로 두고 쉼표로 쓰인 , 만 제거
    w = re.sub(r"\,(?=[0-9])", r"", w)
    
    #마침표로 쓰인 . 만 제거(소수점 . 은 유지)
    w = w[:-1].strip()

    # 모델에 예측 시작과 끝을 알려주기 위해 <start>, <end> 토큰을 추가
    w = '<start> ' + w + ' <end>'
    return w  

In [5]:
#중국어 전처리 
#cut_all=True 로 하면 全模式(완전모드)로, 단어가 될 수 있는 모든 어구를 스캔해 출력
#cut_all=False 로 하면 精确模式 (기본 모드)를 사용해 문장의 변형 없이 단어 단위로 분할 출력
"""
(전)          有人指出,节日家务劳动只集中在女性身上的现象是"性别歧视"
(후)  <start> 有人 指出 , 节日 家务劳动 只 集中 在 女性 身上 的 现象 是 " 性别歧视 " <end>
"""
def preprocess_ch(w):
    w = ' '.join(jieba.cut(w, cut_all=False))   
    w = w[:-1].rstrip().strip()    #문장의 마침표 제거
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
#한국어, 중국어 전처리를 한번에 해줄 함수 작성
def preprocess(path):
    files = glob.glob(os.path.join(path, '*.csv'))
    ch, ko = [], []
    
    for f in files:
        df = pd.read_csv(f)
        ch.extend(df['중국어'].values)
        ko.extend(df['한국어'].values)
    
    ch_series = pd.Series(ch)
    ko_series = pd.Series(ko)
    
    df = pd.concat([ch_series, ko_series], axis=1)
    df.columns = ['중국어', '한국어']
    
    df['중국어'] = df['중국어'].apply(preprocess_ch)
    df['한국어'] = df['한국어'].apply(preprocess_kr)
    
    return df

In [7]:
path = os.getcwd() + '\\Training'
train_df = preprocess(path)

print('훈련 데이터셋 크기 : {}'.format(train_df.shape))
train_df.sample(3)

C:\Users\yeonok\AppData\Local\Temp\ipykernel_14920\4155401409.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yeonok\AppData\Local\Temp\jieba.cache
Loading model cost 0.430 seconds.
Prefix dict has been built successfully.


훈련 데이터셋 크기 : (2080000, 2)


,중국어,한국어
1168411,"<start> 据 赫尔瓦 尔 记者 透露 , 郑灿成 去 洗手间 的 时候 , 奥尔特加 ...",<start> 헬와니 기자에 따르면 정찬성이 화장실에 간 사이 오르테가가 다가와 '...
190993,<start> 现代 汽车 将 提供 自主 开发 的 LBS 平台 和 过去 10 年间 积...,"<start> 현대차는 자체 개발한 LBS 플랫폼과, 지난 10여년 간 축적한 위치..."
793349,"<start> 机器人 核心 零部件 领域 不是 短期投资 就 能 成功 的 领域 , 如果...","<start> 로봇 핵심 부품분야는 단기간 투자로 성공할 수 있는 분야가 아니라, ..."


In [8]:
#전처리가 잘 됐는지 확인
for i in range(10):
    print(train_df.iloc[i,0])
    print(train_df.iloc[i,1])
    print()

<start> 李 会长 对 他 的 公司 计划 购买 沃尔沃 或 萨博 的 报道 上 表示 了 贬值 <end>
<start> 리 회장은 그의 회사가 볼보나 사브를 살 계획이라는 보도에 대해 평가절하했다 <end>

<start> 现代 汽车 相关 人士 表示 : " 追加 混合 动力 车款 的 现代 汽车 科纳 是 拥有 汽油 - 柴油 - 混合 动力 - 电动汽车 国内 最 多 发动机 生产线 的 小型 SUV , 有望 满足 多种 顾客 的 喜爱 。 <end>
<start> 현대차 관계자는 " 하이브리드 모델이 추가된 현대차 코나는 가솔린 - 디젤 - 하이브리드 - 전기차에 이르는 국내 최다 엔진 라인업을 갖춘 소형 SUV로 다양한 고객의 선호를 만족시킬 수 있을 것으로 기대된다 " 고 말했다 <end>

<start> 金融 监管局 最近 下令 制定 一项 计划 , 将 违反 交通法规 的 驾驶员 的 保险费 提高 至 30 ％ , 以 引起 驾驶员 和 政界人士 的 强烈 反对 , 并 由 保险 发展 协会 和 非 人寿保险 业 迅速 制定 改善 计划 <end>
<start> 금감원은 최근 교통법규 위반 운전자에 대해 보험료를 최고 30% 할증하는 방안이 운전자와 정치권의 반발을 사자 보험개발원과 손해보험업계에 조속히 개선안을 마련할 것을 주문했다 <end>

<start> 此前 , 韩国 信用 评级 机构 在 前一天 将 现代 、 起亚 汽车 的 信用等级 分别 下调 了 一个 等级 <end>
<start> 앞서 한신평은 전날 현대 · 기아차의 신용등급을 한단계씩 하향 조정했다 <end>

<start> IHS   Markit 经济学家 肖恩 · 琼斯 表示 ： " 回答 问卷 的 企业 们 强调 了 汽车产业 面临 的 艰难 环境 。 <end>
<start> IHS 마킷 이코노미스트 샨 존스는 " 설문에 답한 업체들이 자동차 산업이 맞닥뜨린 힘든 환경을 강조했다 " 고 전했다 <end>

<start> 驾驶员 和 汽车 制造商 均 可 从 EPB 的 应用 中 获益 <end>
<start> EPB 적용으로 운전자와 자동차제조사 모

In [9]:
train_df.to_csv('train_df.csv')